---
<h1 style="text-align: center;">IF867 - Introdução à Aprendizagem Profunda</h1>
<h2 style="text-align: center;">1ª atividade prática</h2>

---
---
*Discente:*

* Gabriel D'assumpção de Carvalho - gdc2@cin.ufpe.br

*Curso:*

* Ciências Atuariais - 7º Período

<div style="text-align: center;">
29/11/2024
</div>

# Instruções e Requisitos
- Objetivo: Implementar e treinar um Multilayer Perceptron (MLP), inteiramente em [NumPy](https://numpy.org/doc/stable/) ou [Numba](https://numba.readthedocs.io/en/stable/index.html), sem o uso de bibliotecas de aprendizado profundo.
- A atividade pode ser feita em dupla.

## Tarefas

__Implementação (50%):__

- Construa um MLP com uma camada de entrada, pelo menos duas camadas ocultas e uma camada de saída.
- Implemente pelo menos duas funções de ativação diferentes para as camadas ocultas; use Sigmoid e Linear para a camada de saída.
- Implemente forward e backpropagation.
- Implemente um otimizador de sua escolha, adequado ao problema abordado.
- Implemente as funções de treinamento e avaliação.

__Aplicação (30%):__

  Teste se os seus modelos estão funcionando bem com as seguintes tarefas:
  - Regressão
  - Classificação binária

__Experimentação (20%):__

  Teste os seus modelos com variações na arquitetura, no pré-processamento, etc. Escolha pelo menos uma das seguintes opções:
  - Variações na inicialização de pesos
  - Variações na arquitetura
  - Implementação de técnicas de regularização
  - Visualização das ativações e gradientes

***Bônus:*** Implemente o MLP utilizando uma biblioteca de machine learning (ex.: [PyTorch](https://pytorch.org/), [TensorFlow](https://www.tensorflow.org/?hl=pt-br), [tinygrad](https://docs.tinygrad.org/), [Jax](https://jax.readthedocs.io/en/latest/quickstart.html)) e teste-o em uma das aplicações e em um dos experimentos propostos. O bônus pode substituir um dos desafios de aplicação ou experimentos feitos em NumPy, ou simplesmente somar pontos para a pontuação geral.

## Datasets recomendados:
Aqui estão alguns datasets recomendados, mas fica a cargo do aluno escolher os datasets que utilizará na atividade, podendo escolher um dataset não listado abaixo.
- Classificação

  - [Iris](https://archive.ics.uci.edu/dataset/53/iris)
  - [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)
  - [CDC Diabetes Health Indicators](https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators)

- Regressão

  - [Air Quality](https://archive.ics.uci.edu/dataset/360/air+quality)
  - [Student Performance](https://archive.ics.uci.edu/dataset/320/student+performance)
  - [Wine Quality](https://archive.ics.uci.edu/dataset/186/wine+quality)

## Requisitos para Entrega

Um notebook Jupyter (de preferência, o link do colab) ou script Python contendo:

- Código: Implementação completa da MLP.
- Gráficos e Análises: Gráficos da curva de perda, ativações, gradientes e insights do treinamento, resultantes dos experimentos com parada antecipada e diferentes técnicas de regularização.
- Relatório: Um breve relatório detalhando o impacto de várias configurações de hiperparâmetros(ex.: inicialização de pesos, número de camadas ocultas e neurônios) e métodos de regularização no desempenho do modelo.


# Importações

## Bibliotecas

In [1]:
from ucimlrepo import fetch_ucirepo # Database
import numpy as np # Data manipulation
import pandas as pd  # Data manipulation
import matplotlib.pyplot as plt # Data visualization

## Datasets

### Wine Quality

In [ ]:
# Urls of the datasets
url_red = "https://raw.githubusercontent.com/gabrieldadcarvalho/deep_learning/refs/heads/master/exercise_list/01/data/wine/winequality-white.csv"
url_white = "https://raw.githubusercontent.com/gabrieldadcarvalho/deep_learning/refs/heads/master/exercise_list/01/data/wine/winequality-white.csv"

# Read of the arquives CSV
red = pd.read_csv(url_red, sep=";")
white = pd.read_csv(url_white, sep=";")

# Adding a column with the color of the wine
red["color"] = 1
white["color"] = 0

# Concatenation of the datasets
wine = pd.concat([red, white], axis=0)

# 5 first rows
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1


## Iris Dataset

In [10]:
# Url of the iris dataset
url_iris = "https://raw.githubusercontent.com/gabrieldadcarvalho/deep_learning/refs/heads/master/exercise_list/01/data/iris/iris.data"

# Read of the arquives CSV
iris = pd.read_csv(url_iris, sep=",", header=None)

# Adding name of the variables
iris.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# 5 first rows
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Tratamento dos dados

Nessa sessão vai ser elaborado funções para fazer um breve tratamento no banco de dados para facilitar o aprendizado da rede.

sepal_length    0
sepal_width     0
petal_length    0
dtype: int64

In [54]:
iris[iris.duplicated()]

,sepal_length,sepal_width,petal_length,petal_width,species
34,4.9,3.1,1.5,0.1,Iris-setosa
37,4.9,3.1,1.5,0.1,Iris-setosa
142,5.8,2.7,5.1,1.9,Iris-virginica


In [ ]:
# Function to print statistics
def print_stats(df, action, missing=False):
    if missing:
        print(f"Variables with missing values: \n{df.isna().sum()}")
        print("==" * 20)
        print(f"Total of missing values: \n{df.isna().sum().sum()}")
        print("==" * 20)
        if df.isna().any().any(): 
            print(f"Observations of missing values: \n{df[df.isna().any(axis=1)]}")
            print("==" * 20)
    else:
        print(f"Variables with duplicated values: \n{df.duplicated().sum()}")
        print("==" * 20)
        print(f"Total of duplicated values: \n{df.duplicated().sum()}")
        print("==" * 20)
        if df.duplicated().any():  
            print(
                f"Observations of duplicated values: \n{df[df.duplicated(keep=False)]}"
            )
            print("==" * 20)

    print(f"Old shape: {df.shape}")

    return df


# Function for removing missing values
def drop_nan(df):
    # Show statistics
    print_stats(df, action="dropna", missing=True)

    # Drop of missing values
    df_cleaned = df.dropna()
    print(f"New shape: {df_cleaned.shape}")
    return df_cleaned


# Function for removing duplicated values
def drop_duplicates(df):
    # Show Statistics
    print_stats(df, action="drop_duplicates", missing=False)

    # Drop of duplicated values
    df_cleaned = df.drop_duplicates()
    print(f"New shape: {df_cleaned.shape}")
    return df_cleaned